In [1]:
# parallel selection sort


In [6]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git


  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-x7ef0edc
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-x7ef0edc
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=791de4c2ce6defedab9f68b15b131b62e759b5f105b521b073d80f40ecffdb87
  Stored in directory: /tmp/pip-ephem-wheel-cache-qwidqhe1/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [7]:
%load_ext nvcc_plugin 

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [8]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
__global__ void parallel_kernel_pos(int *a, int n)
{
 int i = threadIdx.x;
 int data = a[i];
 int pos = 0;
 for (int j = 0; j < n; j++)
 {
 if (a[j] < data || (a[j] == data && j < i))
 pos++;
 }
 a[pos] = data;
}
void sortArr(int* h_arr, int n){
 int* d_arr = NULL;
 int size = n*sizeof(int);
 cudaError_t err = cudaSuccess;
 err = cudaMalloc((void **)&d_arr,size);
 if(err != cudaSuccess){
 printf("%s\n",cudaGetErrorString(err));
 exit(EXIT_FAILURE);
 }
 err = cudaMemcpy(d_arr,h_arr,size,cudaMemcpyHostToDevice);
 if(err != cudaSuccess){
 printf("%s\n",cudaGetErrorString(err));
 exit(EXIT_FAILURE);
 }
 parallel_kernel_pos<<<1,n>>>(d_arr,n);
 cudaMemcpy(h_arr,d_arr,size,cudaMemcpyDeviceToHost);
 cudaFree(d_arr);
}
int main(){
 int n = 10;
 int* arr = (int*)malloc(n*sizeof(int));
 for(int i=0;i<n;i++){
 arr[i] = rand()%50;
 }
 for(int i=0;i<10;i++){
 printf("%d ",arr[i]);
 }
 //after sorting 
 sortArr(arr,n);
 printf("\n");
 for(int i=0;i<10;i++){
 printf("%d ",arr[i] );
 }
 return 0;
}

33 36 27 15 43 35 36 42 49 21 
33 36 27 15 43 35 36 42 49 21 
